## import libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

In [3]:
tickers=pd.read_csv(r'C:\Users\DELL\.vscode\code\data\top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


## Weighting score model to find final score

In [4]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    ## we have got our df with dividend information
    
    numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [5]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,38.0,5.0,9.779999,44.0,2.4,0.029757,0.028777,0.053588,0.041989,0.169019
1,TCS.NS,168.0,58.0,43.220000,128.0,-1.7,0.146979,0.410072,0.260276,0.134807,0.162047
2,HDFCBANK.NS,115.0,22.0,21.110001,67.0,6.0,0.099188,0.151079,0.123617,0.067403,0.175140
3,INFY.NS,426.0,63.0,64.440000,211.0,-12.7,0.379621,0.446043,0.391433,0.226519,0.143343
4,ICICIBANK.NS,71.0,10.0,14.060001,99.0,16.1,0.059513,0.064748,0.080042,0.102762,0.192314
5,HINDUNILVR.NS,206.0,48.0,94.879997,149.0,-3.8,0.181244,0.338129,0.579578,0.158011,0.158476
6,SBIN.NS,172.0,13.7,15.410000,139.0,70.3,0.150586,0.091367,0.088386,0.146961,0.284475
7,BAJFINANCE.NS,40.0,36.0,13.960000,24.0,15.7,0.031560,0.251799,0.079424,0.019890,0.191634
8,BHARTIARTL.NS,44.0,8.0,19.430000,80.0,491.1,0.035167,0.050360,0.113233,0.081768,1.000000
9,ITC.NS,327.0,14.0,85.350000,407.0,-7.5,0.290352,0.093525,0.520675,0.443094,0.152185


In [6]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
0,RELIANCE.NS,38.0,5.0,9.779999,44.0,2.4,0.029757,0.028777,0.053588,0.041989,0.169019,0.050700
1,TCS.NS,168.0,58.0,43.220000,128.0,-1.7,0.146979,0.410072,0.260276,0.134807,0.162047,0.221329
2,HDFCBANK.NS,115.0,22.0,21.110001,67.0,6.0,0.099188,0.151079,0.123617,0.067403,0.175140,0.115690
3,INFY.NS,426.0,63.0,64.440000,211.0,-12.7,0.379621,0.446043,0.391433,0.226519,0.143343,0.341020
4,ICICIBANK.NS,71.0,10.0,14.060001,99.0,16.1,0.059513,0.064748,0.080042,0.102762,0.192314,0.086596
5,HINDUNILVR.NS,206.0,48.0,94.879997,149.0,-3.8,0.181244,0.338129,0.579578,0.158011,0.158476,0.285365
6,SBIN.NS,172.0,13.7,15.410000,139.0,70.3,0.150586,0.091367,0.088386,0.146961,0.284475,0.138966
7,BAJFINANCE.NS,40.0,36.0,13.960000,24.0,15.7,0.031560,0.251799,0.079424,0.019890,0.191634,0.098854
8,BHARTIARTL.NS,44.0,8.0,19.430000,80.0,491.1,0.035167,0.050360,0.113233,0.081768,1.000000,0.159622
9,ITC.NS,327.0,14.0,85.350000,407.0,-7.5,0.290352,0.093525,0.520675,0.443094,0.152185,0.313783


## Sort according dividend score

In [7]:
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
45,IOC.NS,1114.0,15.0,90.669996,734.0,-76.5,1.000000,0.100719,0.553557,0.804420,0.034858,0.595225
30,COALINDIA.NS,804.0,31.6,46.680000,911.0,-6.3,0.720469,0.220144,0.281661,1.000000,0.154225,0.531924
42,HEROMOTOCO.NS,359.0,140.0,55.480000,317.0,1.3,0.319206,1.000000,0.336053,0.343646,0.167148,0.448417
13,HCLTECH.NS,456.0,72.0,84.260005,296.0,-88.1,0.406673,0.510791,0.513938,0.320442,0.015133,0.392549
20,ONGC.NS,710.0,17.5,40.310000,527.0,-16.8,0.635708,0.118705,0.242289,0.575691,0.136371,0.391686
22,POWERGRID.NS,489.0,15.0,70.519996,515.0,-4.2,0.436429,0.100719,0.429013,0.562431,0.157796,0.365141
31,BPCL.NS,524.0,15.5,32.650003,572.0,19.3,0.467989,0.104317,0.194944,0.625414,0.197755,0.345107
41,TATASTEEL.NS,259.0,3.6,162.900000,287.0,-38.1,0.229035,0.018705,1.000000,0.310497,0.100153,0.344566
3,INFY.NS,426.0,63.0,64.440000,211.0,-12.7,0.379621,0.446043,0.391433,0.226519,0.143343,0.341020
28,TECHM.NS,308.0,45.0,89.750000,225.0,76.4,0.273219,0.316547,0.547871,0.241989,0.294848,0.332732
